In [1]:
import pandas as pd
import numpy as np
import math
#모델 라이브러리 선언
from sklearn import datasets, tree
#모델 정확도 라이브러리 선언
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

C:\Users\SMART-14\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
###  kopo_decision_tree_all_new.csv 파일을 열고 판매량을 예측하는 모델을 생성하세요    ###


In [3]:
kopoData = pd.read_csv("./dataset/kopo_decision_tree_all_new.csv")

In [52]:
kopoData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PRO_YN,H_YN
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0,0,0
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0,0,1
2,A01,PG01,P01,ITEM001,201549,2015,49,2,Y,0,N,0.0,0,1
3,A01,PG01,P01,ITEM002,201526,2015,26,1,Y,1,N,0.0,0,1
4,A01,PG01,P01,ITEM002,201532,2015,32,1,N,4,N,0.0,0,0


In [5]:
kopoData.dtypes
kopoData["PRO_YN"] = np.where(kopoData.PROMOTION=="Y",1,0)
kopoData["H_YN"] = np.where(kopoData.HOLIDAY=="Y",1,0)
kopoData.head(2)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PRO_YN,H_YN
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0,0,0
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0,0,1


In [6]:
groupKey = ["REGIONID","PRODUCTGROUP","PRODUCT","ITEM"]

In [37]:
kopoDataCount = kopoData.groupby(groupKey)["YEARWEEK"].\
                agg(["size"]).reset_index()
kopoDataCount.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,size
0,A01,PG01,P01,ITEM001,3
1,A01,PG01,P01,ITEM002,2
2,A01,PG01,P01,ITEM003,1
3,A01,PG01,P01,ITEM004,7
4,A01,PG01,P01,ITEM005,8


In [39]:
kopoDataCount.rename(columns={"size":"KNOB"},inplace=True)
kopoDataCount.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,KNOB
0,A01,PG01,P01,ITEM001,3
1,A01,PG01,P01,ITEM002,2
2,A01,PG01,P01,ITEM003,1
3,A01,PG01,P01,ITEM004,7
4,A01,PG01,P01,ITEM005,8


In [43]:
mergedData = pd.merge(kopoData,kopoDataCount,on = groupKey)
mergedData.head()

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PRO_YN,H_YN,KNOB
0,A01,PG01,P01,ITEM001,201538,2015,38,1,N,4,N,0.0,0,0,3
1,A01,PG01,P01,ITEM001,201548,2015,48,1,Y,0,N,0.0,0,1,3
2,A01,PG01,P01,ITEM001,201549,2015,49,2,Y,0,N,0.0,0,1,3
3,A01,PG01,P01,ITEM002,201526,2015,26,1,Y,1,N,0.0,0,1,2
4,A01,PG01,P01,ITEM002,201532,2015,32,1,N,4,N,0.0,0,0,2


In [41]:
mergedData.KNOB.max()

146

In [44]:
refindData = mergedData[mergedData.KNOB == mergedData.KNOB.max() ]

In [45]:
len(refindData["ITEM"].drop_duplicates())

52

In [46]:
#상관관계 확인
corrdf = kopoData.corr()
corrdf

,YEARWEEK,YEAR,WEEK,QTY,HCLUS,PRO_PERCENT,PRO_YN,H_YN
YEARWEEK,1.000000,0.984528,0.005567,-0.021432,0.016040,0.161164,0.118040,0.001566
YEAR,0.984528,1.000000,-0.169746,-0.039309,0.068345,0.122548,0.097886,-0.041092
WEEK,0.005567,-0.169746,1.000000,0.103788,-0.299818,0.207041,0.105250,0.243307
QTY,-0.021432,-0.039309,0.103788,1.000000,-0.184016,0.237682,0.095984,0.157853
HCLUS,0.016040,0.068345,-0.299818,-0.184016,1.000000,-0.289396,-0.143294,-0.977417
PRO_PERCENT,0.161164,0.122548,0.207041,0.237682,-0.289396,1.000000,0.812344,0.254015
PRO_YN,0.118040,0.097886,0.105250,0.095984,-0.143294,0.812344,1.000000,0.136497
H_YN,0.001566,-0.041092,0.243307,0.157853,-0.977417,0.254015,0.136497,1.000000


In [47]:
groupKey = ["REGIONID","PRODUCTGROUP","PRODUCT","ITEM"]
groupData=refindData.groupby(groupKey)
len(groupData)


52

In [50]:
def mlfunction(onegroup):

    #onegroup = groupData.get_group(list(groupData.groups)[8])

    eachgroup = onegroup.reset_index(drop=True)

    #     1. 상관관계를 산출
    #     2. 데이터를 분리
    #     3. 모델 선언 및 학습
    #     4. 예측
    ##상관관계 산출 후 feature와 lebel 정의 corrdf = eachgroup.corr()

    features=list(corrdf[abs(corrdf.QTY>0.5)&(corrdf.QTY != 1)].index)

    #len(features) == 0 

    label=["QTY"]

    ##데이터 분리

    stdindex=round(len(eachgroup)*0.8)

    stdYearweek=eachgroup.loc[stdindex,["YEARWEEK"]].values[0] 

    stdYearweek = 201700

    trainingData_features = eachgroup[eachgroup.YEARWEEK <= stdYearweek][features]
    trainingData_label = eachgroup[eachgroup.YEARWEEK <= stdYearweek][label]
    testData_features = eachgroup[eachgroup.YEARWEEK > stdYearweek][features]
    testData_label = eachgroup[eachgroup.YEARWEEK > stdYearweek][label]
    testData_label2 = eachgroup[eachgroup.YEARWEEK > stdYearweek]

    ##모델 선언 및 학습

    model_method = tree.DecisionTreeRegressor(random_state=1) 
    model=model_method.fit(trainingData_features,trainingData_label)

    ##예측

    testData_label2["PREDICT"] = model.predict(testData_features)

    return testData_label2

In [51]:
finalData = refindData.groupby(groupKey).apply(mlfunction)

ValueError: Found array with 0 feature(s) (shape=(105, 0)) while a minimum of 1 is required.

In [30]:
finalData.reset_index(drop=True)

NameError: name 'finalData' is not defined

In [18]:
def mlFcst(onegroup): 
    
#     onegroup = groupData.get_group(list(groupData.groups)[index])

#         onegroup = groupData.get_group(list(groupData.groups)[7]) 
        eachgroup = onegroup.reset_index()

    #     1. 상관관계를 산출
    #     2. 데이터를 분리
    #     3. 모델 선언 및 학습
    #     4. 예측
        ##상관관계 산출 후 feature와 lebel 정의 corrdf = eachgroup.corr()

        features=list(corrdf[abs(corrdf.QTY>0.5)&(corrdf.QTY != 1)].index)
        

        label=["QTY"]

        ##데이터 분리

        stdindex=round(len(eachgroup)*0.8)

        stdYearweek=eachgroup.loc[stdindex,["YEARWEEK"]].values[0] 

        trainingData_features = eachgroup[eachgroup.YEARWEEK <= stdYearweek][features]
        trainingData_label = eachgroup[eachgroup.YEARWEEK <= stdYearweek][label]
        testData_features = eachgroup[eachgroup.YEARWEEK > stdYearweek][features]
        testData_label = eachgroup[eachgroup.YEARWEEK > stdYearweek][label]
        testData_label2 = eachgroup[eachgroup.YEARWEEK > stdYearweek]

        ##모델 선언 및 학습

        model_method = tree.DecisionTreeRegressor(random_state=1) 
        model=model_method.fit(trainingData_features,trainingData_label)

        ##예측

        testData_label2["PREDICT"] = model.predict(testData_features)

        return testData_label2

In [19]:
refindData.head(5)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PRO_YN,H_YN,KNOB
2352,A01,PG02,P03,ITEM043,201501,2015,1,87,Y,1,Y,0.197590,1,1,146
2353,A01,PG02,P03,ITEM043,201502,2015,2,60,N,4,Y,0.197590,1,0,146
2354,A01,PG02,P03,ITEM043,201503,2015,3,51,N,4,N,0.000000,0,0,146
2355,A01,PG02,P03,ITEM043,201504,2015,4,37,Y,2,N,0.000000,0,1,146
2356,A01,PG02,P03,ITEM043,201505,2015,5,136,N,4,Y,0.201205,1,0,146


In [20]:
from sklearn.preprocessing import LabelEncoder

le_pro = LabelEncoder()
le_holy =LabelEncoder()

refindData['HOLY_ENCO']= le_pro.fit_transform(refindData.PROMOTION)



C:\Users\SMART-14\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [21]:
refindData.head(5)

,REGIONID,PRODUCTGROUP,PRODUCT,ITEM,YEARWEEK,YEAR,WEEK,QTY,HOLIDAY,HCLUS,PROMOTION,PRO_PERCENT,PRO_YN,H_YN,KNOB,HOLY_ENCO
2352,A01,PG02,P03,ITEM043,201501,2015,1,87,Y,1,Y,0.197590,1,1,146,1
2353,A01,PG02,P03,ITEM043,201502,2015,2,60,N,4,Y,0.197590,1,0,146,1
2354,A01,PG02,P03,ITEM043,201503,2015,3,51,N,4,N,0.000000,0,0,146,0
2355,A01,PG02,P03,ITEM043,201504,2015,4,37,Y,2,N,0.000000,0,1,146,0
2356,A01,PG02,P03,ITEM043,201505,2015,5,136,N,4,Y,0.201205,1,0,146,1
